In [ ]:
import os
import pandas as pd
import urllib
import ffmpeg
import moviepy.editor as mp
import cv2
import io
import PIL
import random
import math
import shutil
import base64
from PIL import Image

In [ ]:
os.chdir("/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_links")
LINK_DIR = "/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_links"
PARENT_DIR = "/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_vids"
df = pd.read_csv("nw_april_v3.csv")

In [ ]:
from google.cloud import videointelligence_v1 as videointelligence
from google.cloud import vision

os.chdir("/nfs/sloanlab003/projects/arc_proj/story/code/nw/google-cloud-sdk")
GOOGLE_APPLICATION_CREDENTIALS = "/nfs/sloanlab003/projects/arc_proj/story/code/nw/ad-story-arc-b60c485322b4.json"
print('done')


new_df = pd.DataFrame()
new_df['id'] = [None]
new_df['video_id'] = [None]
new_df['bounding_box'] = [None]
new_df['emotions'] = [None]
new_df

new_df = pd.DataFrame()
temp_df = pd.DataFrame({
    "id" : "nope",
    "video_id": "slayage",
    "bounding_box": "ur mom",
    "emotions" : "stop"
}, index=[len(new_df)])
#
# Append a dataframe
#
new_df = new_df.append(temp_df)
new_df

## Google Vision API Face Detection


In [ ]:
import io
import PIL
import math
import shutil
import base64
from google.cloud import vision
from google.oauth2 import service_account

os.chdir("/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_links")
PARENT_DIR = "/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_vids"
new_df = pd.DataFrame()

# feed video into Google vision API to see what 
for i in range(3498, df.shape[0]):
    
    if df['has_vid'][i] == 0:
        continue
    
    # store video id and length
    ID = df['id'][i]
    
    # cd into video's directory
    os.chdir(f'{PARENT_DIR}/{ID}/img_frames')
    
    if i % 20 == 0:
        print(f'starting to parse {ID}')
    
    if i % 100 == 0 and i != 3500:
        os.chdir(LINK_DIR)
        new_df.to_csv(f"nw_faceinfo_{i-99}-{i}.csv")
        print(f"saved nw_faceinfo_{i-99}-{i}.csv")
        new_df = pd.DataFrame()
        os.chdir(f'{PARENT_DIR}/{ID}/img_frames')
        
    for filename in os.listdir(f'{PARENT_DIR}/{ID}/img_frames'):

        frame = filename
        frame_id = frame[:-4]
        
        im = PIL.Image.open(frame, 'r')
        bounding_boxes = google_image_face(PARENT_DIR, ID, frame)
        
        # for each face detected, save the bounding box as seperate image
        for j, face_info in enumerate(bounding_boxes):
            
            index = str(j+1)
            x = index.zfill(2)
            
            bounding_box = str(face_info[0])
            emotions = face_info[1]

            temp_df = pd.DataFrame({
                "id" : f"{frame_id}_f{x}",
                "video id": ID,
                "bounding box": bounding_box,
                "emotions" : emotions
            }, index=[len(new_df)])
            
            try:
                new_df = new_df.append(temp_df)
                
            except:
                print('something went wrong')
            
            
            
print('finished')
os.chdir(LINK_DIR)
new_df.to_csv("nw_faceinfo_v1.csv")

## Cloud vision API - Function Definition


In [ ]:
def google_image_face(parent_dir, id_, path):
    
    os.chdir(f'{parent_dir}/{id_}/img_frames')
    from google.oauth2 import service_account
    from google.cloud import vision
    import io

    credentials = service_account.Credentials.from_service_account_file("/nfs/sloanlab003/projects/arc_proj/story/code/nw/ad-story-arc-b60c485322b4.json")
    client = vision.ImageAnnotatorClient(credentials=credentials)

    with io.open(path, 'rb') as image_file:
        content = image_file.read()


    image = vision.Image(content=content)

    response = client.face_detection(image=image)
    faces = response.face_annotations
    

    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')
    
    face_info = []
    
    for face in faces:
        face_labels = ['anger: {}'.format(likelihood_name[face.anger_likelihood]), 'joy: {}'.format(likelihood_name[face.joy_likelihood]), 'surprise: {}'.format(likelihood_name[face.surprise_likelihood]), 
                  'sorrow: {}'.format(likelihood_name[face.sorrow_likelihood]), 'under_exposed: {}'.format(likelihood_name[face.under_exposed_likelihood]), 
                  'blurred: {}'.format(likelihood_name[face.blurred_likelihood]), 'headwear: {}'.format(likelihood_name[face.headwear_likelihood])]
        

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in face.bounding_poly.vertices])
        
        bounding_box = crop(face)       
        face_info.append([bounding_box, '\n'.join(face_labels)])
        

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
        
    return face_info

In [ ]:
CROP_MARGIN_PERCENT = 10

def crop(face_annotation):
    v = face_annotation.bounding_poly.vertices
    x1, y1, x2, y2 = v[0].x, v[0].y, v[2].x + 1, v[2].y + 1
    w, h = x2 - x1, y2 - y1
    hx, hy = x1 + w / 2, y1 + h / 2
    m = max(w, h) * (100 + CROP_MARGIN_PERCENT) / 100 / 2
    return int(hx - m + 0.5), int(hy - m + 0.5), int(hx + m + 1.5), int(hy + m + 1.5)

## Cloud Video Intelligence API

In [ ]:
def detect_person(local_file_path):
    """Detects people in a video from a local file."""
    from google.oauth2 import service_account
    from google.cloud import vision
    import io

    credentials = service_account.Credentials.from_service_account_file("/nfs/sloanlab003/projects/arc_proj/story/code/nw/ad-story-arc-b60c485322b4.json")
    
    client = videointelligence.VideoIntelligenceServiceClient(credentials=credentials)

    with io.open(local_file_path, "rb") as f:
        input_content = f.read()

    # Configure the request
    config = videointelligence.FaceDetectionConfig(
        include_bounding_boxes=True, include_attributes=True
    )
    context = videointelligence.VideoContext(face_detection_config=config)

    # Start the asynchronous request
    operation = client.annotate_video(
        request={
            "features": [videointelligence.Feature.FACE_DETECTION],
            "input_content": input_content,
            "video_context": context,
        }
    )

    print("\nProcessing video for face detection annotations.")
    result = operation.result(timeout=300)

    print("\nFinished processing.\n")

    # Retrieve the first result, because a single video was processed.
    annotation_result = result.annotation_results[0]
    
    return_list = []

    for annotation in annotation_result.face_detection_annotations:
        face_info = []
        
        # print("Face detected:")
        for track in annotation.tracks:

            timestamped_object = track.timestamped_objects[0]
            box = timestamped_object.normalized_bounding_box
            
            
            face_info.append({"box": (box.left, box.top, box.right, box.bottom)})
            face_info.append({"start_time": track.segment.start_time_offset.seconds + track.segment.start_time_offset.microseconds / 1e6})
            face_info.append({"end_time": track.segment.end_time_offset.seconds + track.segment.end_time_offset.microseconds / 1e6})



            return_list.append(face_info)
    # print('here', return_list)
    return return_list

In [ ]:
def crop_from_video(ID, bounding_box, timestamp):
    os.chdir(f'{PARENT_DIR}/{ID}')
    filename = ID + '.mp4'
    
    vidcap = cv2.VideoCapture(filename)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    success,image = vidcap.read()
    count = 0
    success = True

    while success:
        success,frame = vidcap.read()

        count+=1
        if count/fps >= timestamp:
            # print('frame #', count, 'time', count/fps)
            imageRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            im = Image.fromarray(imageRGB)
            roi = crop_normalized(bounding_box, im)
            
            vidcap.release()
            cv2.destroyAllWindows()
            return roi
            break
            
    vidcap.release()
    cv2.destroyAllWindows()


In [ ]:
def crop_normalized(bounding_box, image):
    left, top, right, bottom = bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3]
    number_of_rows, number_of_columns = image.height, image.width
    
    top_unormalized = round(top * number_of_rows)
    left_unormalized = round(left * number_of_columns)
    bottom_unormalized = round(bottom * number_of_rows)
    right_unormalized = round(right * number_of_columns)
    
    return image.crop((left_unormalized, top_unormalized, right_unormalized, bottom_unormalized))

## Running videos through Video Intelligence API
Saves faces as cropped frames

In [ ]:
# run videos through video intelligence API
os.chdir(LINK_DIR)
df = pd.read_csv('temp_1500-2000.csv')

num_faces_save = 10
df2 = df.copy()

df_num = 1

for i in range(df.shape[0]):
    
    if df["has_vid"][i] == 0:
        continue
    
    ID = df["id"][i]
    os.chdir(f'{PARENT_DIR}/{ID}')
    
    try:
        result = detect_person(ID+".mp4")
        segment_length = {}
        os.mkdir(f"{PARENT_DIR}/{ID}/google_video_face_samples")
    
        for j, face in enumerate(result[::-1]):
            bb = face[0]['box']
            start_time = face[1]['start_time']
            end_time = face[2]['end_time']
            segment_length[end_time - start_time] = (bb, start_time)

        num_faces = len(segment_length.keys())
        top_faces = sorted(segment_length.keys(), reverse=True)[:num_faces_save]

        df2['faces detected'][i] = num_faces

        if i % 500 == 0 and i != 0:
            os.chdir(LINK_DIR)
            df2.to_csv(f'temp_{i-500}-{i}.csv', index=False)


        for j, segment in enumerate(top_faces):
            (bb, start_time) = segment_length[segment]
            roi = crop_from_video(ID, bb, start_time)
            roi.save(f"{PARENT_DIR}/{ID}/google_video_face_samples/{str(j).zfill(3)}_{str(round(start_time, 2)).zfill(5)}.jpg")

        print('done with', ID)
    except:
        didnt_work.append(ID)
        print(ID, "didn't work")
        
    
os.chdir(LINK_DIR)
df2.to_csv('TEST.csv', index=False)
print('done!')

In [ ]:
# run 10 sample videos through google vision API
os.chdir(f"{LINK_DIR}")
df = pd.read_csv("nw_april_faceinfo_ALL.csv")
for ID in id_test:
    os.chdir(f'{PARENT_DIR}/{ID}/img_frames')
    counter = 0
    for i in range(df.shape[0]):
        os.chdir(f'{PARENT_DIR}/{ID}/img_frames')
        if df["video id"][i] != ID:
            continue
        
        frame_path = df["id"][i][:-4]+".jpg"
        print(frame_path)
        im = PIL.Image.open(frame_path, 'r')
        bounding_box = tuple(map(int, df["bounding box"][i][1:-1].split(', ')))
        roi = im.crop(bounding_box)
        
        os.chdir(f"{LINK_DIR}/google_vision_face_samples")
        roi.save(f"{frame_path}_face_{counter}.jpg")
        counter += 1
        
    
print("done")